In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import copy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
torch.cuda.empty_cache()
torch.cuda.reset_max_memory_allocated()
torch.cuda.synchronize()






Using device: cuda:0


c:\Users\anton\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\cuda\memory.py:391: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


In [2]:
print(torch.cuda.is_available())  # Should print True
print(torch.cuda.device_count())  # Should be > 0
print(torch.cuda.get_device_name(0))  # Prints your GPU name

True
1
NVIDIA GeForce RTX 2050


## TASK 1


In [ ]:


# Define the transform to convert images to tensor and normalize them
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Download and load the training dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=4)

# Download and load the testing dataset
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=4)




In [7]:


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.relu = nn.LeakyReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 16 * 16, 10)  # Assuming 32x32 input images
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x


In [10]:


# Ensure device is set to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Initialize TensorBoard writer
writer = SummaryWriter()
print("TensorBoard writer created")

# Initialize model, loss function, and optimizer
model = Net().to(device)
criterion = nn.CrossEntropyLoss().to(device)  # Ensure loss function is on GPU
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)

train_loss = []
test_loss_array = []
best_accuracy = 0.0

num_epochs = 15
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    model.train()
    running_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for i, (inputs, labels) in enumerate(train_loader):
        print("Batch", i+1, end='\r')
        inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, predicted = torch.max(outputs, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    train_loss.append(running_loss / len(train_loader))
    train_accuracy = 100 * train_correct / train_total

    # Log training loss and accuracy
    writer.add_scalar('Loss/Train', train_loss[-1], epoch)
    writer.add_scalar('Accuracy/Train', train_accuracy, epoch)

    # Testing
    model.eval()
    test_loss = 0.0
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

        test_loss /= len(test_loader)
        test_accuracy = 100 * test_correct / test_total
        test_loss_array.append(test_loss)

        # Log test loss and accuracy
        writer.add_scalar('Loss/Test', test_loss, epoch)
        writer.add_scalar('Accuracy/Test', test_accuracy, epoch)

        # Save the best model based on test accuracy
        if epoch == 0 or test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, 'cloudclass.pth')

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss[-1]:.2f} | Train Accuracy: {train_accuracy:.2f}% | Test Loss: {test_loss:.2f} | Test Accuracy: {test_accuracy:.2f}%")

print(f"Best test accuracy: {best_accuracy:.2f}%")

# Close the TensorBoard writer
writer.close()


Using device: cuda
TensorBoard writer created
Epoch 1/15
Epoch 1/15 | Train Loss: 1.92 | Train Accuracy: 32.93% | Test Loss: 1.76 | Test Accuracy: 36.82%
Epoch 2/15
Epoch 2/15 | Train Loss: 1.73 | Train Accuracy: 40.48% | Test Loss: 1.76 | Test Accuracy: 37.91%
Epoch 3/15


KeyboardInterrupt: 